In [1]:
from sklearn.metrics import adjusted_mutual_info_score as AMI 
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
from collections import Counter
import igraph as ig
import pandas as pd
import numpy as np
import h_louvain as hl
import csv
import os
import random
import matplotlib.pyplot as plt
import h_louvain_BO as hlbo
import pickle


In [6]:
def build_habcd():
    cmd = julia + ' -n ' + str(nodes)
    cmd = cmd + ' -d ' + str(deg_exp) + ',' + str(deg_min) + ',' + str(deg_max)
    cmd = cmd + ' -c ' + str(com_exp) + ',' + str(com_min) + ',' + str(com_max)
    cmd = cmd + ' -x ' + str(xi)
    cmd = cmd + ' -q ' + str(edge_sizes[0])
    for i in np.arange(1,len(edge_sizes),1):
        cmd = cmd + ',' + str(edge_sizes[i])
    cmd = cmd + ' -w wcd.txt'
    cmd = cmd + ' -s ' + str(seed)
    cmd = cmd + ' -o hgwcdM >& _temp'     
    return cmd

In [3]:
def load_ABCDH_from_file(filename):
    with open(filename,"r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(set(line))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    return HG

In [4]:
def get_AMI(HG,gt,A):
    d = hmod.part2dict(A)
    A4ami = [d[str(i+1)] for i in range(len(HG.nodes))]
    return AMI(gt, A4ami)


In [5]:
julia = '/Applications/Julia-1.7.app/Contents/Resources/julia/bin/julia --project /Users/francois/ABCD/ABCDHypergraphGenerator/utils/abcdh.jl'

nodes = 300
deg_min = 5
deg_max = 30
deg_exp = 2.5
com_min = 40
com_max = 60
com_exp = 1.5
edge_sizes = [0,0,0,0,1]
#edge_type = 'majority'

# ## FCT - experimental modularity function


In [9]:
## different graphs
xi = 0.2
L = []
def expo(d, c):
    return (c/d)**(3) if c > d/2 else 0  
for rep in range(30):
    seed = rep
    random.seed(rep)
    cmd = build_habcd()
    os.system(cmd)

    with open("./hgwcdM_he.txt","r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(set(line))

    with open("./hgwcdM_assign.txt", 'r') as file:
        gt = [int(line) for line in file]
    gt_dict = {str(i+1):gt[i] for i in range(len(gt))}

    mc = max(gt)
    Nodes = [x for x in gt_dict.keys() if gt_dict[x]>=(mc-1)]
    for i in range(35):
        Edges.append(set(np.random.choice(Nodes,5,replace=False)))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    
    hL = hl.hLouvain(HG, hmod_type=expo, delta_it = 0.0001, delta_phase = 0.0001, random_seed = seed) 
    A_gt = [x for x in hmod.dict2part(gt_dict) if len(x)>0]
    qH_gt = hL.combined_modularity(A_gt, expo, 1)

    ## two-section
    G = hnx.two_section(HG)
    dct = hnx.part2dict(A_gt)
    for v in G.vs:
        v['gt'] = dct[v['name']]   
    qH_ts = -1
    for ts_rep in range(5):
        ml = G.community_multilevel(weights='weight').membership
        q = hL.combined_modularity(hnx.dict2part(dict(zip(G.vs['name'], ml))), expo, 0)
        if q > qH_ts:
            ami_ts = AMI(G.vs['gt'],ml)
            qH_ts = q 
    V = [ami_ts]
    for ex in [0,1,2,3,4,5,6,7,8]:
        def expo(d, c):
            return (c/d)**(ex/2) if c > d/2 else 0  
        hBO = hlbo.hLouvainBO(HG, hmod_type=expo, delta_it=0.0001, delta_phase=0.0001, random_seed = seed)
        hBO.set_params(hmod_type=expo, show_bo_table=False,seeds=[rep,rep+100,rep+200, rep+300, rep+400])
        df_bo = hBO.hLouvain_perform_BO()
        qH_bo = hL.combined_modularity(df_bo["A"].tolist()[0], expo, 1)
        ami_bo = get_AMI(HG, gt, df_bo["A"].tolist()[0])
        V.append(ami_bo)
    L.append(V)
    print(V)
    fn = 'test_more_5.pkl'
    with open(fn, 'wb') as fp:
        pickle.dump(L, fp)


[0.7643569765943058, 0.4711452969729837, 0.6749186311005512, 0.8230989134015044, 0.7091805342387776, 0.6276587105178152, 0.7732608865331462, 0.6692590463780373, 0.5232892536262361, 0.49054005689483243]
[0.7405811350554304, 0.6410386664289568, 0.7432229338301504, 0.8004522009823235, 0.8887031130456234, 0.8776306350114441, 0.7258374818868373, 0.6680499163838138, 0.6931298279202605, 0.5922673408042445]
[0.8572275930517308, 0.7121166122376836, 0.5923028693393274, 0.7492036033705747, 0.7962868992900486, 0.8303698012937739, 0.7677020581887484, 0.7774147269996743, 0.7285328142628471, 0.6109354128847404]
[0.684732154487903, 0.604614357024284, 0.7430539140868672, 0.7955093176119586, 0.76264235507386, 0.812237321521907, 0.7037662311323691, 0.6483024530896598, 0.5190553167766345, 0.5199356710108967]
[0.7994843520070442, 0.5776233275317708, 0.7355054180583553, 0.8028316592387625, 0.8291894898654831, 0.8686674877294942, 0.820063392774383, 0.7134622894121334, 0.6715749385643778, 0.5007209808769071]


In [11]:
## same graph
xi = 0.2
L = []
def expo(d, c):
    return (c/d)**(3) if c > d/2 else 0  

seed = 12345
random.seed(12345)
cmd = build_habcd()
os.system(cmd)

with open("./hgwcdM_he.txt","r") as f:
    rd = csv.reader(f)
    lines = list(rd)
Edges = []
for line in lines:
    Edges.append(set(line))

with open("./hgwcdM_assign.txt", 'r') as file:
    gt = [int(line) for line in file]
gt_dict = {str(i+1):gt[i] for i in range(len(gt))}

mc = max(gt)
Nodes = [x for x in gt_dict.keys() if gt_dict[x]>=(mc-1)]
for i in range(35):
    Edges.append(set(np.random.choice(Nodes,5,replace=False)))

HG = hnx.Hypergraph(dict(enumerate(Edges)))
A_gt = [x for x in hmod.dict2part(gt_dict) if len(x)>0]
G = hnx.two_section(HG)
dct = hnx.part2dict(A_gt)
for v in G.vs:
    v['gt'] = dct[v['name']]   

for rep in range(30):
    seed = rep
    random.seed(rep)

    hL = hl.hLouvain(HG, hmod_type=expo, delta_it = 0.0001, delta_phase = 0.0001, random_seed = seed) 
    qH_gt = hL.combined_modularity(A_gt, expo, 1)

    ## two-section
    qH_ts = -1
    for ts_rep in range(5):
        ml = G.community_multilevel(weights='weight').membership
        q = hL.combined_modularity(hnx.dict2part(dict(zip(G.vs['name'], ml))), expo, 0)
        if q > qH_ts:
            ami_ts = AMI(G.vs['gt'],ml)
            qH_ts = q 
    V = [ami_ts]
    for ex in [0,1,2,3,4,5,6,7,8]:
        def expo(d, c):
            return (c/d)**(ex/2) if c > d/2 else 0  
        hBO = hlbo.hLouvainBO(HG, hmod_type=expo, delta_it=0.0001, delta_phase=0.0001, random_seed = seed)
        hBO.set_params(hmod_type=expo, show_bo_table=False,seeds=[rep,rep+100,rep+200, rep+300, rep+400])
        df_bo = hBO.hLouvain_perform_BO()
        qH_bo = hL.combined_modularity(df_bo["A"].tolist()[0], expo, 1)
        ami_bo = get_AMI(HG, gt, df_bo["A"].tolist()[0])
        V.append(ami_bo)
    L.append(V)
    print(V)
    fn = 'test_more_5_fixed.pkl'
    with open(fn, 'wb') as fp:
        pickle.dump(L, fp)


[0.723847940920294, 0.6642862345530417, 0.8027445791887888, 0.7911622605152598, 0.8488532326790647, 0.666026513081996, 0.654490526242815, 0.6022523321255678, 0.6825418615219268, 0.5844058391250828]
[0.7346271115070496, 0.8452422759763775, 0.6467156482108883, 0.7941917064107609, 0.8588722476741737, 0.8171026119979768, 0.6937864853160528, 0.6344693947401957, 0.628357285887066, 0.5916159266246078]
[0.7322799713919913, 0.698852146843813, 0.8386783812513117, 0.8663222188656698, 0.8466947065734702, 0.8519713869435994, 0.7100515305818007, 0.6229793397379626, 0.6394573297625268, 0.5557562418354189]
[0.7644895383476595, 0.4717231446542987, 0.7411062735980509, 0.7894951009611935, 0.8260056191109816, 0.8623521736089866, 0.7438485296799123, 0.6677976676102091, 0.5661605900973382, 0.5694289698452429]
[0.7288100734300498, 0.49217069050740864, 0.5460251570881532, 0.718297646725091, 0.7644407704410079, 0.7750247464654406, 0.7358972601802662, 0.6292350173989724, 0.5642893043160571, 0.5827472373512863]
